In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Brugada_Syndrome"
cohort = "GSE136992"

# Input paths
in_trait_dir = "../../input/GEO/Brugada_Syndrome"
in_cohort_dir = "../../input/GEO/Brugada_Syndrome/GSE136992"

# Output paths
out_data_file = "../../output/preprocess/Brugada_Syndrome/GSE136992.csv"
out_gene_data_file = "../../output/preprocess/Brugada_Syndrome/gene_data/GSE136992.csv"
out_clinical_data_file = "../../output/preprocess/Brugada_Syndrome/clinical_data/GSE136992.csv"
json_path = "../../output/preprocess/Brugada_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"mRNA expression in SIDS"
!Series_summary	"Genetic predispositions in cases suffering sudden unexpected infant death have been a research focus worldwide the last decade. Despite large efforts there is still uncertainty concerning the molecular pathogenesis of these deaths. With genetic technology in constant development the possibility of an alternative approach into this research field have become available, like mRNA expression studies.  Methods: In this study we investigated mRNA gene expression in 14 cases that died suddenly and unexpectedly from infection without a history of severe illness prior to death. The control group included eight accidents, two cases of natural death, one undetermined, one case of medical malpractice and two homicides. The study included tissue from liver, heart and brain. The mRNA expression was determined using Illumina whole genome gene expression DASL HT assay.  Results: From the array, 19 genes showed altered ex

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
is_gene_available = True  # Based on the background information which mentions "mRNA expression studies" and "Illumina whole genome gene expression DASL HT assay"

# 2.1 and 2.2 Data Availability and Type Conversion

# For trait - Brugada Syndrome
# The condition field (row 0) has 'Infection' and 'Control' values
# Since KCNE5 downregulation in infection cases is associated with Brugada syndrome, 
# we can use the 'condition' field to determine Brugada syndrome risk
trait_row = 0

def convert_trait(value):
    # Extract the value after the colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Map 'Infection' to 1 (higher risk of Brugada syndrome due to KCNE5 downregulation)
    # and 'Control' to 0
    if value.lower() == 'infection':
        return 1
    elif value.lower() == 'control':
        return 0
    else:
        return None

# For age
age_row = 2

def convert_age(value):
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Extract the numeric part (age in weeks)
        age_value = float(value.split()[0])
        return age_value  # Return age as a continuous variable
    except (ValueError, IndexError):
        return None

# For gender
gender_row = 3

def convert_gender(value):
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'male':
        return 1
    elif value.lower() == 'female':
        return 0
    else:
        return None

# 3. Save metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Initial filtering of dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    try:
        # Process clinical features using the actual clinical_data from previous steps
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # Use the variable from previous steps
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview processed clinical data
        preview = preview_df(selected_clinical_df)
        print("Clinical Features Preview:")
        print(preview)
        
        # Save the processed clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except NameError:
        print("Error: clinical_data variable not found. Make sure clinical data is loaded in a previous step.")
    except Exception as e:
        print(f"Error processing clinical features: {str(e)}")


Clinical Features Preview:
{'GSM4064970': [1.0, 24.0, 1.0], 'GSM4064971': [1.0, 112.0, 1.0], 'GSM4064972': [1.0, 8.0, 0.0], 'GSM4064973': [1.0, 24.0, 1.0], 'GSM4064974': [1.0, 0.6, 0.0], 'GSM4064975': [1.0, 72.0, 1.0], 'GSM4064976': [1.0, 24.0, 0.0], 'GSM4064977': [1.0, 36.0, 1.0], 'GSM4064978': [1.0, 52.0, 1.0], 'GSM4064979': [1.0, 20.0, 1.0], 'GSM4064980': [0.0, 24.0, 0.0], 'GSM4064981': [0.0, 0.0, 0.0], 'GSM4064982': [0.0, 0.0, 0.0], 'GSM4064983': [0.0, 80.0, 0.0], 'GSM4064984': [0.0, 52.0, 0.0], 'GSM4064985': [0.0, 0.5, 0.0], 'GSM4064986': [0.0, 144.0, 1.0], 'GSM4064987': [0.0, 0.0, 1.0], 'GSM4064988': [0.0, 24.0, 1.0], 'GSM4064989': [0.0, 0.0, 1.0], 'GSM4064990': [1.0, 112.0, 1.0], 'GSM4064991': [1.0, 24.0, 1.0], 'GSM4064992': [1.0, 8.0, 0.0], 'GSM4064993': [1.0, 0.6, 0.0], 'GSM4064994': [1.0, 20.0, 1.0], 'GSM4064995': [1.0, 36.0, 1.0], 'GSM4064996': [1.0, 12.0, 0.0], 'GSM4064997': [1.0, 72.0, 1.0], 'GSM4064998': [1.0, 0.0, 0.0], 'GSM4064999': [1.0, 52.0, 1.0], 'GSM4065000': [1.0,

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Brugada_Syndrome/GSE136992/GSE136992_family.soft.gz
Matrix file: ../../input/GEO/Brugada_Syndrome/GSE136992/GSE136992_series_matrix.txt.gz
Found the matrix table marker at line 60


Gene data shape: (29377, 60)
First 20 gene/probe identifiers:
['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286', 'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315']


### Step 4: Gene Identifier Review

In [5]:
# Analyzing gene identifiers
# The ILMN_ prefix indicates these are Illumina probe IDs, not standard human gene symbols
# Illumina BeadArray technologies use these proprietary identifiers which need to be mapped to standard gene symbols
# Therefore, gene mapping is required

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Examine the columns to find gene information
print("\nExamining gene mapping columns:")
print("Column 'ID' examples (probe identifiers):")
id_samples = gene_annotation['ID'].head(5).tolist()
for i, sample in enumerate(id_samples):
    print(f"Example {i+1}: {sample}")

# Look at Symbol column which contains gene symbols
print("\nColumn 'Symbol' examples (contains gene symbols):")
if 'Symbol' in gene_annotation.columns:
    # Display a few examples of the Symbol column
    symbol_samples = gene_annotation['Symbol'].head(10).tolist()
    for i, sample in enumerate(symbol_samples):
        print(f"Example {i+1}: {sample}")
    
    # Extract some gene symbols to verify
    print("\nChecking if symbols are proper human gene symbols:")
    human_gene_count = 0
    total_symbols = 50  # Check a few more rows
    symbol_samples_extended = gene_annotation['Symbol'].dropna().head(total_symbols).tolist()
    for i, sample in enumerate(symbol_samples_extended[:10]):  # Show first 10 examples
        symbols = extract_human_gene_symbols(str(sample))
        is_human = bool(symbols) and symbols[0] == sample
        human_gene_count += int(is_human)
        print(f"Example {i+1}: '{sample}' - {'Likely human gene' if is_human else 'Not standard human gene'}")
    
    print(f"\nOut of {total_symbols} examined symbols, {human_gene_count} appear to be standard human gene symbols.")
    
    # Identify the columns needed for gene mapping
    print("\nColumns identified for gene mapping:")
    print("- 'ID': Contains probe IDs (e.g., ILMN_3166687)")
    print("- 'Symbol': Contains gene symbols (e.g., ERCC-00162)")
else:
    print("Error: 'Symbol' column not found in annotation data.")

# Check if there are other columns that might contain human gene information
print("\nChecking other columns that might contain gene information:")
potential_gene_columns = ['ILMN_Gene', 'Entrez_Gene_ID', 'Synonyms']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nExamples from '{col}' column:")
        examples = gene_annotation[col].dropna().head(5).tolist()
        for i, ex in enumerate(examples):
            print(f"Example {i+1}: {ex}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Transcript', 'Species', 'Source', 'Search_Key', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009'], 'Source_Reference_ID': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009'], 'RefSeq_ID': [nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan], 'GI': [nan, nan, nan], 'Access

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene mapping
# From the previous output, we can see:
# - 'ID' column contains the probe identifiers (ILMN_xxxx) which match the gene identifiers in the gene expression data
# - 'Symbol' column contains gene symbols we want to map to

# 2. Create a gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of the mapping dataframe:")
print(mapping_df.head())

# Check overlap between probes in expression data and mapping data
genes_in_expr = set(gene_data.index)
genes_in_mapping = set(mapping_df['ID'])
overlap = genes_in_expr.intersection(genes_in_mapping)
print(f"\nOverlap between expression data and mapping data: {len(overlap)} probes")
print(f"Percentage of expression probes with mapping: {len(overlap)/len(genes_in_expr)*100:.2f}%")

# 3. Apply gene mapping to convert probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"\nGene-level expression data shape: {gene_data.shape}")
print("First few rows of gene expression data:")
print(gene_data.head())

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (29377, 2)
First few rows of the mapping dataframe:
             ID        Gene
0  ILMN_3166687  ERCC-00162
1  ILMN_3165566  ERCC-00071
2  ILMN_3164811  ERCC-00009
3  ILMN_3165363  ERCC-00053
4  ILMN_3166511  ERCC-00144

Overlap between expression data and mapping data: 29377 probes
Percentage of expression probes with mapping: 100.00%



Gene-level expression data shape: (20211, 60)
First few rows of gene expression data:
         GSM4064970    GSM4064971    GSM4064972   GSM4064973    GSM4064974  \
Gene                                                                         
A1BG      62.355348     10.278570     54.362789     5.764988     23.992323   
A1CF      43.321260     22.707244     16.152246    24.781712     55.479592   
A26C3      5.361441      5.217641      5.870840     9.111313      5.317085   
A2BP1  23878.363768  12539.905276  20798.931157  6185.899282  18210.533055   
A2LD1     22.563890     31.984639     11.114187     5.945264     17.456945   

         GSM4064975  GSM4064976    GSM4064977    GSM4064978    GSM4064979  \
Gene                                                                        
A1BG      31.366236  267.552914   1879.845723    112.182100     78.592842   
A1CF      16.285685   26.136836    862.133248     22.251610     92.120753   
A26C3      5.510478   59.801096      5.260670      4.43841

Gene expression data saved to ../../output/preprocess/Brugada_Syndrome/gene_data/GSE136992.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data is available before trying to extract features
if trait_row is not None:
    # Extract clinical features only if trait_row exists
    clinical_features = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    
    # Handle missing values and check for bias
    linked_data = handle_missing_values(linked_data, trait)
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
else:
    # No clinical data available
    print("No clinical data available for the trait of interest.")
    linked_data = pd.DataFrame()  # Empty dataframe
    is_biased = True  # Dataset is biased since we have no trait data

# 6. Validate and save cohort info
is_trait_available = trait_row is not None
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data but lacks COVID-19 trait information."
)

# 7. Save the linked data if it's usable
if is_usable:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save the linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved due to quality issues.")

Gene data shape after normalization: (19450, 60)


Normalized gene data saved to ../../output/preprocess/Brugada_Syndrome/gene_data/GSE136992.csv
Clinical features saved to ../../output/preprocess/Brugada_Syndrome/clinical_data/GSE136992.csv


For the feature 'Brugada_Syndrome', the least common label is '0.0' with 29 occurrences. This represents 48.33% of the dataset.
The distribution of the feature 'Brugada_Syndrome' in this dataset is fine.

Quartiles for 'Age':
  25%: 0.375
  50% (Median): 24.0
  75%: 52.0
Min: 0.0
Max: 144.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 30 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Brugada_Syndrome/cohort_info.json


Linked data saved to ../../output/preprocess/Brugada_Syndrome/GSE136992.csv
